## Gesture Recognition System

In [25]:
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
from numpy.fft import rfft, rfftfreq
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import json
from scipy.stats import kurtosis

def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    # Create a list of DataFrames, one for each gesture
    dfs = []        
    for item in data:
        df = pd.DataFrame({
            'timestamps': item['timestamps'],
            'xTimeSeries': item['xTimeSeries'],
            'yTimeSeries': item['yTimeSeries'],
            'zTimeSeries': item['zTimeSeries'],
            'label': item['label']
        })
        dfs.append(df)
    return dfs

### Signal preprocessing

In [26]:
def preprocess_signal_bandpass(data, lowcut=5.0, highcut=25.0, fs=50, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    y = filtfilt(b, a, data)
    return y


### Feature extraction

In [27]:

def extract_features(segment):
    features = []
    for axis in ['xTimeSeries', 'yTimeSeries', 'zTimeSeries']:
        axis_data = segment[axis]
        # Time domain features
        features += [
            np.mean(axis_data),
            np.std(axis_data),
            np.min(axis_data),
            np.max(axis_data),
            np.sum(np.abs(axis_data)) / len(axis_data),  # SMA
            np.sum(np.abs(axis_data)**2),  # Energy
            np.sum(np.diff(np.sign(axis_data)) != 0),  # Zero crossings
            kurtosis(axis_data)  # Kurtosis
        ]
        # Frequency domain features
        spectral_energy, dominant_frequency = extract_frequency_features(axis_data)
        features += [spectral_energy, dominant_frequency]
    return features


def extract_frequency_features(data):
    fft_vals = rfft(data)
    fft_freq = rfftfreq(len(data))
    spectral_energy = np.sum(np.abs(fft_vals)**2) / len(fft_vals)
    dominant_frequency = fft_freq[np.argmax(np.abs(fft_vals))]
    return spectral_energy, dominant_frequency


### Training

In [28]:
def train_classifier(features, labels):
    model = make_pipeline(StandardScaler(), SVC())
    model.fit(features, labels)
    return model

def predict_with_confidence(model, features, threshold = 0.95):
    decision_values = model.decision_function([features])
    prediction = model.predict([features])
    confidence = np.abs(decision_values[0])  # Use absolute value for confidence
    if confidence < threshold:
        return "No Gesture", confidence  # Return empty label if below threshold
    else:
        return prediction[0], confidence  # Return prediction and confidence otherwise

In [29]:
import matplotlib.pyplot as plt

def plot_preprocessed_data(dataframes):
    for i, df in enumerate(dataframes):
        # Original data
        if i < 3:
            plt.figure(figsize=(15, 5))
            plt.subplot(1, 2, 1)
            plt.plot(df['timestamps'], df['xTimeSeries'], label='Original X', color='red')
            plt.plot(df['timestamps'], df['yTimeSeries'], label='Original Y', color='green')
            plt.plot(df['timestamps'], df['zTimeSeries'], label='Original Z', color='blue')
            plt.title('Original Accelerometer Data')
            plt.xlabel('Time')
            plt.ylabel('Acceleration')
            plt.legend()

        # Preprocessed data
        df['xTimeSeries'] = preprocess_signal(df['xTimeSeries'])
        df['yTimeSeries'] = preprocess_signal(df['yTimeSeries'])
        df['zTimeSeries'] = preprocess_signal(df['zTimeSeries'])
        if i < 3:
            plt.subplot(1, 2, 2)
            plt.plot(df['timestamps'], df['xTimeSeries'], label='Preprocessed X', color='red')
            plt.plot(df['timestamps'], df['yTimeSeries'], label='Preprocessed Y', color='green')
            plt.plot(df['timestamps'], df['zTimeSeries'], label='Preprocessed Z', color='blue')
            plt.title('Preprocessed Accelerometer Data')
            plt.xlabel('Time')
            plt.ylabel('Acceleration')
            plt.legend()
            plt.tight_layout()
            plt.show()


### In-Practice Usage

In [30]:
from sklearn.model_selection import train_test_split

# Load gesture data
dataframes = load_data('labeled_train_data/labeled_data_circle.json') + load_data('labeled_train_data/labeled_data_clap.json') 
random_dataframes = load_data('labeled_train_data/labeled_data_random.json')

print("")
# Plot preprocessed data for visual inspection (if preprocess is defined and required here)
# plot_preprocessed_data(dataframes)

# Prepare features and labels for gesture data
features = []
labels = []
for df in dataframes:
    # Preprocess the data here if necessary, e.g., df['xTimeSeries'] = preprocess_signal(df['xTimeSeries'])
    features.append(extract_features(df))
    labels.append(df['label'].iloc[0])  # Assuming all entries in a df have the same label

# Split the gesture data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Prepare features and labels for random data, and use them only in the testing phase
random_features = []
random_labels = []
for df in random_dataframes:
    # Preprocess the random data here if necessary
    random_features.append(extract_features(df))
    random_labels.append('No Gesture')  # Label all random movements as 'No Gesture'

# Combine the test set with the random data
features_test.extend(random_features)
labels_test.extend(random_labels)

# Train the classifier on the training set
model = train_classifier(features_train, labels_train)

# Evaluate the classifier on the testing set (now includes random data)
predictions_test = []
confidences_test = []
for feature in features_test:
    prediction, confidence = predict_with_confidence(model, feature)
    predictions_test.append(prediction)
    confidences_test.append(confidence)

for i, tup in enumerate(zip(predictions_test, labels_test)):
    print(f"Predicted: {tup[0]}, Actual: {tup[1]}, with confidence: {confidences_test[i]:.2f}")

# Calculate accuracy on the testing data including random data
accuracy_test = sum(1 for p, l in zip(predictions_test, labels_test) if p == l) / len(labels_test)
print(f"Accuracy on test data including random movements: {accuracy_test:.2f}")





Predicted: Circle, Actual: Circle, with confidence: 1.14
Predicted: Clap, Actual: Clap, with confidence: 1.10
Predicted: Circle, Actual: Circle, with confidence: 1.14
Predicted: Circle, Actual: Circle, with confidence: 1.32
Predicted: Circle, Actual: Circle, with confidence: 1.20
Predicted: Clap, Actual: Clap, with confidence: 1.09
Predicted: Circle, Actual: Circle, with confidence: 1.24
Predicted: No Gesture, Actual: Clap, with confidence: 0.88
Predicted: Clap, Actual: Clap, with confidence: 1.06
Predicted: Circle, Actual: Circle, with confidence: 1.28
Predicted: Circle, Actual: Circle, with confidence: 1.13
Predicted: Clap, Actual: Clap, with confidence: 1.28
Predicted: Clap, Actual: Clap, with confidence: 0.98
Predicted: Clap, Actual: Clap, with confidence: 1.18
Predicted: No Gesture, Actual: Clap, with confidence: 0.45
Predicted: Circle, Actual: Circle, with confidence: 1.23
Predicted: No Gesture, Actual: Circle, with confidence: 0.86
Predicted: Clap, Actual: Clap, with confidence

### Conversion to ONNX

In [31]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

feature_count = len(features_train[0])  # Replace this with the actual number of features if known
initial_type = [('float_input', FloatTensorType([None, feature_count]))]

onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("gesture_classifier.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [42]:
import onnxruntime as rt
import numpy as np

def onnx_predict_with_confidence(onnx_model_path, features, threshold=1):
    # Load the ONNX model
    sess = rt.InferenceSession(onnx_model_path)
    input_name = sess.get_inputs()[0].name
    output_label = sess.get_outputs()[0].name  # The predicted label output
    output_probability = sess.get_outputs()[1].name  # The probabilities output

    # Run the model (ensure features is a numpy array and correctly shaped)
    probabilities = sess.run([output_probability], {input_name: np.array([features], dtype=np.float32)})[0]
    predictions = sess.run([output_label], {input_name: np.array([features], dtype=np.float32)})[0]
    
    # Calculate confidence (the highest class probability)
    max_prob = np.max(probabilities)
    prediction = predictions[0]

    if max_prob < threshold:
        return "No Gesture", max_prob
    else:
        return prediction, max_prob

# Example usage
predictions_test = []
confidences_test = []
for feature in features_test:
    prediction, confidence = onnx_predict_with_confidence("gesture_classifier.onnx",feature)
    predictions_test.append(prediction)
    confidences_test.append(confidence)

for i, tup in enumerate(zip(predictions_test, labels_test)):
    print(f"Predicted: {tup[0]}, Actual: {tup[1]}, with confidence: {confidences_test[i]:.2f}")

# Calculate accuracy on the testing data including random data
accuracy_test = sum(1 for p, l in zip(predictions_test, labels_test) if p == l) / len(labels_test)
print(f"Accuracy on test data including random movements: {accuracy_test:.2f}")



Predicted: Circle, Actual: Circle, with confidence: 1.14
Predicted: Clap, Actual: Clap, with confidence: 1.10
Predicted: Circle, Actual: Circle, with confidence: 1.14
Predicted: Circle, Actual: Circle, with confidence: 1.32
Predicted: Circle, Actual: Circle, with confidence: 1.20
Predicted: Clap, Actual: Clap, with confidence: 1.09
Predicted: Circle, Actual: Circle, with confidence: 1.24
Predicted: No Gesture, Actual: Clap, with confidence: 0.88
Predicted: Clap, Actual: Clap, with confidence: 1.06
Predicted: Circle, Actual: Circle, with confidence: 1.28
Predicted: Circle, Actual: Circle, with confidence: 1.13
Predicted: Clap, Actual: Clap, with confidence: 1.28
Predicted: No Gesture, Actual: Clap, with confidence: 0.98
Predicted: Clap, Actual: Clap, with confidence: 1.18
Predicted: No Gesture, Actual: Clap, with confidence: 0.45
Predicted: Circle, Actual: Circle, with confidence: 1.23
Predicted: No Gesture, Actual: Circle, with confidence: 0.86
Predicted: Clap, Actual: Clap, with confi